## Air-Quality-in-Durban-Part-1-EDA-with-MongoDB by Dumisani Maxwell Mukuchura

#### Dataset Source: *https://open.africa/dataset/sensorsafrica-airquality-archive-durban*

##### Contact: dumisanimukuchura@gmail.com | LinkedIn: https://www.linkedin.com/in/dumisani-maxwell-mukuchura-4859b7170/

##### This Project will explore the time series dataset for Air Quality and will dissect it with Exploratory Data Analysis 

Kindly note this was done from a Local machine on Windows with a locally hosted database using Mongo DB, Mongo DB Compass and MongoDB Shell i.e a NoSQL database.  

In [2]:
#Import Libraries

from pprint import PrettyPrinter  #makes documents easier to read
import pandas as pd 
from pymongo import MongoClient  #useful to get Data from Mongo DB   

## 1. Prepare Data

### 1.1. Connect

In [6]:
pp = PrettyPrinter(indent = 2)
client = MongoClient(host= "localhost", port = 27017)

### 1.2. Explore

In [9]:
# What Databases are available?
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'admin', 'sizeOnDisk': 40960},
  {'empty': False, 'name': 'air_quality', 'sizeOnDisk': 49152},
  {'empty': False, 'name': 'config', 'sizeOnDisk': 110592},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 40960}]


In [10]:
#Assign database to db variable
db = client["air_quality"]

In [11]:
# View available collections in the db - start with a single one to understand the form
list(db.list_collections())[0]

{'name': 'durban',
 'type': 'collection',
 'options': {},
 'info': {'readOnly': False,
  'uuid': Binary(b'\xea\x8d\xa6{q\x94@\xed\xbc\xa5\xd8\x83\xdb\x00;\xcb', 4)},
 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}

In [12]:
# View available collections in the db
list(db.list_collection_names())

['durban']

In [13]:
# Assign the Durban collection to durban variable
durban = db["durban"]

In [15]:
#Count how many documents are in the Durban Collection
document_count = durban.count_documents({})
document_count

424

In [16]:
#View one document to understand the structure
pp.pprint(durban.find_one({}))

{ '_id': ObjectId('67c70bc3f6ce00cb6195a489'),
  'lat': -29.848,
  'location': 1,
  'lon': 30.922,
  'sensor_id': 21,
  'sensor_type': 'SDS011',
  'timestamp': datetime.datetime(2017, 11, 24, 14, 21, 55, 531000),
  'value': 5.0,
  'value_type': 'P2'}


In [18]:
#Determining how many distinct type of readings were done in the Durban collection
durban.distinct("value_type")

['P1', 'P2', 'humidity', 'temperature']

In [22]:
#Determining how many distinct type of readings were done in the Durban collection
durban.distinct("sensor_type")

['SDS011']

In [19]:
#Determine how many documents are there by Value Type?
agg_result = durban.aggregate(
    [
        {"$group": {"_id": "$value_type", "count":{"$count":{}}}}
    ]
)
pp.pprint(list(agg_result))

[ {'_id': 'humidity', 'count': 52},
  {'_id': 'temperature', 'count': 52},
  {'_id': 'P1', 'count': 160},
  {'_id': 'P2', 'count': 160}]


## 2. Import

In [24]:
#Using projection to build to Dataframe Import - focusing on P2 readings only.

result = durban.find(
    {"value_type": "P2"},
    projection = {"value": 1, "timestamp": 1, "_id": 0} #set id to zero as we do not need it to appear, value of P2 and timestamp to appear
)
pp.pprint(result.next())

{'timestamp': datetime.datetime(2017, 11, 24, 14, 21, 55, 531000), 'value': 5.0}


In [25]:
#read the result into a DataFrame
df = pd.DataFrame(result).set_index("timestamp")
df.head()

,value
timestamp,
2017-11-24 17:13:18.557,13.07
2017-11-24 17:27:16.687,13.50
2017-11-24 17:35:29.732,13.60
2017-11-24 17:46:49.968,11.30
2017-11-24 17:54:23.458,11.00
